In [ ]:
pip install requests pandas

In [ ]:
import requests
import pandas as pd

# URL para buscar todos os jogos do Campeonato Paulista 2025
url_jogos = 'https://www.sofascore.com/api/v1/unique-tournament/83/season/53786/events'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36'
}

# Fazendo a requisição para obter todos os jogos do Campeonato Paulista
response = requests.get(url_jogos, headers=headers)

if response.status_code == 200:
    jogos_data = response.json()
    all_data = []

    print("\n🔎 Buscando estatísticas dos jogos do Campeonato Paulista 2025...\n")

    for jogo in jogos_data.get('events', []):  # Garante que 'events' existe antes de iterar
        match_id = jogo.get('id')
        home_team = jogo.get('homeTeam', {}).get('name', 'Desconhecido')
        away_team = jogo.get('awayTeam', {}).get('name', 'Desconhecido')

        print(f"📌 Coletando dados de {home_team} x {away_team} (ID: {match_id})")

        # URL para buscar estatísticas específicas de cada jogo
        url_stats = f'https://www.sofascore.com/api/v1/event/{match_id}/statistics'
        response_stats = requests.get(url_stats, headers=headers)

        if response_stats.status_code == 200:
            stats_data = response_stats.json()

            for match_stats in stats_data.get('statistics', []):
                period = match_stats.get('period', 'Desconhecido')

                for group in match_stats.get('groups', []):
                    group_name = group.get('groupName', 'Sem grupo')

                    for item in group.get('statisticsItems', []):
                        stat_name = item.get('name', 'Sem estatística')
                        home_value = item.get('home', 0)
                        away_value = item.get('away', 0)

                        all_data.append({
                            'Match ID': match_id,
                            'Home Team': home_team,
                            'Away Team': away_team,
                            'Period': period,
                            'Group': group_name,
                            'Stat Name': stat_name,
                            'Home Value': home_value,
                            'Away Value': away_value
                        })
        else:
            print(f"⚠️ Erro ao obter estatísticas para {home_team} x {away_team}: {response_stats.status_code}")

    # Criar um DataFrame e exportar para Excel
    if all_data:
        df = pd.DataFrame(all_data)
        excel_file = 'paulista_2025_stats.xlsx'
        df.to_excel(excel_file, index=False)
        print(f"\n✅ Dados exportados com sucesso para '{excel_file}'!")
    else:
        print("\n❌ Nenhum dado foi coletado.")

else:
    print(f"❌ Erro ao obter a lista de jogos do campeonato: {response.status_code}")

In [ ]:
df